In [1]:
#import rbsa
import pandas as pd
import numpy as np
import os
import datetime
from calendar import monthrange
import matplotlib.pyplot as plt

In [3]:
"""Load electrification
"""

import sys
import pandas as pd

def find_csvfile(name):
    return str(sys.modules[__name__].__file__).replace(f"/loads.py",f"/{name}.csv")


electrification_index = ["location"]
electrification_data = ["heat_pump","other_electric_heat","cooling","water_heating","cooking"]
electrification_attributes = ["city","region","building_type"]
def load_electrification (
        select={},
        index="location",
        columns=["city","region","building_type","heat_pump",
            "other_electric_heat","cooling","water_heating","cooking"],
        convert=pd.DataFrame,
        version='latest'
        ):

    data = pd.read_csv('C:/Users/Palash Goiporia/GISMO_files/load_composition_analysis/src/electrification/loads-latest.csv')
    for key,value in select.items():
        data = data[data[key]==value]
    return convert(data.set_index(index)[columns])

locations = pd.read_csv("rbsa_locations.csv")
locations['postcode'] = np.array(locations['postcode'])//100
siteid_zip = {}
for i in range(locations.shape[0]):
    siteid_zip[str(locations.iloc[i][0])] = locations.iloc[i][3]


FileNotFoundError: [Errno 2] No such file or directory: 'rbsa_locations.csv'

In [ ]:
#pd.set_option('display.max_rows', None)
electrification = load_electrification()
electrification

In [ ]:
import feeder_composition as fc
x = fc.rbsa_file_reader('weekday', electrification)

In [ ]:
rbsa_data_weekday = x
rbsa_enduses = None
for col in rbsa_data_weekday[1].keys():
    try:
        rbsa_enduses = pd.merge(rbsa_data_weekday[1][col].drop(columns = ['temp']), rbsa_enduses, left_index = True, right_index = True)
    except:
        rbsa_enduses = rbsa_data_weekday[1][col]
rbsa_enduses.to_csv('rbsa_enduses.csv')

In [ ]:
import load_composition as lc
start = datetime.datetime.now()
dt = lambda ts: datetime.datetime.strptime(ts,"%Y-%m-%d %H:%M:%S")
rbsa_enduses = pd.read_csv('rbsa_enduses.csv')
rbsa_enduses['index'] = [dt(t) for t in rbsa_enduses.rename(columns = {'Unnamed: 0': 'Date'}).set_index('Date').index.values]
rbsa_enduses = rbsa_enduses.set_index('index').drop(columns = ['Unnamed: 0'])
sens = lc.sensitivity_models(buildings = None, data = rbsa_enduses, weekday = 'weekday', build_class = 'residential')
print((datetime.datetime.now() - start).total_seconds())

In [ ]:
import feeder_composition as fc
start = datetime.datetime.now()
old = 0
commercial_composite_enduses = {}
#building_type_dict = {}
for xls in sorted(os.listdir('C:/Users/Palash Goiporia/GISMO_files/ceus_data/xls/')):
    if xls.endswith('.xls'):
        filename = xls.split('.')[0]
        zone, building_type = filename.split('_')[0], filename.split('_')[1]
        df = fc.composite_enduse(filename, electrification)
        if zone != old:
            building_type_dict = {}

        building_type_dict[building_type] = df
        try:
            commercial_composite_enduses[zone].append(building_type_dict)
        except:
            commercial_composite_enduses[zone] = building_type_dict
        old = zone
        print(filename)
print((datetime.datetime.now() - start).total_seconds())

In [ ]:
area_dict = {}
building_dict = {}
for zone in commercial_composite_enduses.keys():
    for build in commercial_composite_enduses[zone].keys():
        try:
            building_dict[build] = building_dict[build].append(commercial_composite_enduses[zone][build][1])

        except:
            building_dict[build] = commercial_composite_enduses[zone][build][1]

        try:
            for col in commercial_composite_enduses[zone][build][3].keys():
                area_dict[build][col] += commercial_composite_enduses[zone][build][3][col]
        except:
            y = {}
            for col in commercial_composite_enduses[zone][build][3].keys():
                y[col] = commercial_composite_enduses[zone][build][3][col]
            area_dict[build] = y

In [ ]:
normed_load = {}
for b in building_dict.keys():
    temp = building_dict[b].fillna(0).groupby('hour').mean()['drybulb']
    x = []
    for t in building_dict[b]['hour']:
        x.append(pd.Timestamp(t))
    building_dict[b]['hour'] = x
    v = building_dict[b].fillna(0).groupby('hour').sum()
    for col in area_dict[b].keys():
        v[col] = np.array(v[col])/area_dict[b][col]
    v['drybulb'] = temp
    normed_load[b] = v

In [ ]:
start = datetime.datetime.now()
ceus_tester = lc.sensitivity_models(normed_load.keys(), normed_load, 'weekday', 'commercial')
print((datetime.datetime.now() - start).total_seconds())

In [ ]:
import load_composition as lc
import sensitivity_new as sn
loc = 'MKE'
rbsa_tester = sens
weather = lc.weather_season(loc, 'weekday')
test = lc.comp_enduses(weather = weather, ceus_sens = ceus_tester, rbsa_sens = rbsa_tester, location = loc, feeder = 'commercial', electrification = electrification, debug = True)

#ceus_comp_enduses(weather = weather, ceus_sens = tester, rbsa_sens = sens, location = loc, feeder = 'commercial')